In [1]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments
from datasets import load_dataset

from dotenv import load_dotenv
import os 

load_dotenv()
os.environ['HF_DATASETS_OFFLINE'] = '0'
os.environ['TRANSFORMERS_OFFLINE'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
WORK_DIR = os.path.join(os.environ['WORK_DIR'], 'tw_llama_tags')

2023-12-11 13:08:59.942124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 13:08:59.942188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 13:09:00.085773: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 13:09:00.185672: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
auto_args =lambda x: x.from_pretrained('yentinglin/Taiwan-LLM-7B-v2.1-chat', 
                                       torch_dtype=torch.float16, 
                                       token=os.environ['HUGGINGFACE_TOKEN'],
                                       device_map='auto')

tokenizer, model = auto_args(AutoTokenizer), auto_args(AutoModelForCausalLM)

model.gradient_checkpointing_enable()

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right' # Fix weird overflow issue with fp16 training

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model.device

device(type='cuda', index=0)

In [4]:
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.float16, device_map='auto')

# # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
# messages = [
#     {
#         "role": "system",
#         "content": "你是一個人工智慧助理",
#     },
#     {   
#         "role": "user", 
#         "content": "東北季風如何影響台灣氣候？"
#     },
# ]
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
# print(outputs[0]["generated_text"])

In [5]:
dataset = load_dataset('csv', data_files='tags.csv', split='train')
dataset

Dataset({
    features: ['reviews', 'summarization'],
    num_rows: 177
})

In [8]:
def generate_text(example):
    # if example['summarization'] == None: 
    #     example['summarization'] = ''
        
    proc_str = ''.join([
        # '你是一個人工智慧助理,',
        '你的任務是根據以下幾個對某間餐廳的評論，',
        '以100字以內總結這些評論。',
        '主要針對餐廳的特點去做摘要，不要把有關餐廳地點的論述包括進來。',
        '</s>USER:',
        '\n\n'.join(eval(example['reviews'])).strip(), 
        '</s>ASSISTANT:', 
        example['summarization']
    ])
    
    return {
        'reviews': example['reviews'],
        'summarization': example['summarization'],
        'text': proc_str,
    }
    
def get_processed_dataset():
    return (
        dataset.shuffle(seed=42)
               .map(generate_text)
        # .remove_columns()
    )
    
dataset = get_processed_dataset()
dataset

In [11]:
from peft import LoraConfig, TaskType
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(r=16, task_type="CAUSAL_LM")

# Set training parameters
training_args = TrainingArguments(
    output_dir=os.path.join(WORK_DIR, 'llama_tags'),
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    save_steps=-1,
    logging_steps=50,
    fp16=True,
    save_safetensors=True,
    # report_to='tensorboard',
    seed=42,
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_args,
    dataset_text_field='text',
    max_seq_length=2048,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:418: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,1.449000
100,1.317900


TrainOutput(global_step=110, training_loss=1.3744730776006526, metrics={'train_runtime': 1073.2952, 'train_samples_per_second': 0.825, 'train_steps_per_second': 0.102, 'total_flos': 4.686042217439232e+16, 'train_loss': 1.3744730776006526, 'epoch': 4.97})

In [26]:
pipe = pipeline('text-generation', 
                model=model, 
                tokenizer=tokenizer, 
                torch_dtype=torch.float16, 
                device_map='auto',
                return_full_text=False)

review = dataset['reviews'][3]

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        'role': 'system',
        'content': ''.join(['你的任務是根據以下幾個對某間餐廳的評論，',
                            '以100字以內總結這些評論。',
                            '主要針對餐廳的特點去做摘要，不要把有關餐廳地點的論述包括進來。',
                            ]),        
    },
    {   
        'role': 'user', 
        'content': '\n\n'.join(eval(review)).strip()
    },
]

prompt = pipe.tokenizer.apply_chat_template(messages, 
                                            tokenize=False, 
                                            add_generation_prompt=True)

# outputs = pipe(prompt, 
#                max_new_tokens=300, 
#                do_sample=True, 
#                temperature=0.7, 
#                top_k=50, 
#                top_p=0.95)

outputs = pipe(prompt, 
               max_new_tokens=300, 
               do_sample=True, 
               temperature=0.6, 
               top_k=50, 
               top_p=0.95)

print(outputs[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:418: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:59: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


 這家餐廳的招牌炸豬排咖哩飯價格雖貴，但味道十分不錯，滑蛋也很好吃。雖然飯量不夠，但整體來說是值得一吃的。餐廳位於寶山路巷子內，咖哩飯偏甘甜，沒有辣味，豬排炸得恰到好處。價格因人而異，看大家能否接受。


In [22]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear(
                in_featur

In [23]:
from peft import AutoPeftModelForCausalLM
trained_model = AutoPeftModelForCausalLM.from_pretrained(os.path.join(WORK_DIR, 'llama_tags'),
                                                         low_cpu_mem_usage=True,)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
merged_model = trained_model.merge_and_unload()
merged_model.save_pretrained(save_directory=os.path.join(WORK_DIR, 'llama_tags_merged'), 
                             safe_serialization=True)

In [2]:
auto_args =lambda x, y: x.from_pretrained(y, 
                                          torch_dtype=torch.float16, 
                                          device_map='auto')

model = auto_args(AutoModelForCausalLM, os.path.join(WORK_DIR, 'llama_tags_merged'))
tokenizer = auto_args(AutoTokenizer, 'yentinglin/Taiwan-LLM-7B-v2.1-chat')

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]